In [5]:
import drms
import json, numpy as np, matplotlib.pylab as plt, matplotlib.ticker as mtick
import urllib
from astropy.io import fits
from skimage.transform import resize

In [2]:
import drms
c = drms.Client()
c.series(r'hmi\.sharp_')
# Set a series
si = c.info('hmi.sharp_cea_720s')

#### You only need to change the date and path in the name in the cells below

In [3]:
date = '2010.01.01 - 2014.01.01'    #change this to your desired time period
keys = c.query('hmi.sharp_cea_720s[][{}][? (CRLN_OBS < 25) AND (CRLT_OBS < 25) AND (USFLUX > 4e21) ?]'.format(date), key='T_REC, HARPNUM, USFLUX, CRLT_OBS, CRLN_OBS, AREA_ACR')

In [6]:
harpnums = keys['HARPNUM'].unique()
for harpnum in harpnums:
    i = np.where(keys['HARPNUM'] == harpnum)[0]
    t_rec = keys['T_REC'][i]
    t_rec = t_rec[::8]   #we select every 8th element, so we have time intervals of 12*8=96 minutes as of right now
    for t in t_rec:
        hmi_query_string = 'hmi.sharp_cea_720s[{}][{}]'.format(harpnum, t)
        keys_temp, segments = c.query(hmi_query_string, key='T_REC, USFLUX, ERRVF', seg='Br')
        url = 'http://jsoc.stanford.edu' + segments.Br[0]  
        photosphere_image = fits.open(url)                  
        img = photosphere_image[1].data
        res = resize(img, (128, 128))
        name = t + '_{}'.format(harpnum)
        
        name = name.replace(':', '_')
        name = name.replace('.', '_')
        

        path = '/Volumes/L7aler_HD 1/ADL_Project/AR_data/new_SHARP_Data/2010-2014/{}.npy'.format(name)   #change this to your path
        with open(path, 'wb') as f:
            np.save(f, res)

        
    